In [2]:
import fastText

#### Generate a data set

In [49]:
with open('./names.words') as file:
    names = file.readlines()

with open('./lastnames.words') as file:
    last_names = file.readlines()

with open('./credits.words') as file:
    credits_words = file.readlines()
    
with open('./words.words') as file:
    english = file.readlines()

In [92]:
import random

credits_training_size = 3000
not_training_size = 1000
credits_max_sentence_length = 9
not_max_sentence_length = 50
credits_training = []
non_credits_training = []

def is_credits_related(word):
    for w in credits_words:
        if any(word.lower() in credits_term for credits_term in w.lower().split(' ')):
            return True
    
    return False

for i in range(credits_training_size):
    sentence = ''
    length = int(random.random() * credits_max_sentence_length + 1)
    
    for j in range(length):
        title = random.choice(credits_words).lower()
        first = random.choice(names).lower()
        last = random.choice(last_names).lower()
        sentence += title + ' ' + first + ' ' + last + ' '
        sentence = sentence.replace('\n', '')
    credits_training.append(sentence)

for i in range(not_training_size):
    sentence = ''
    length = int(random.random() * not_max_sentence_length + 1)
    
    for j in range(length):
        word = random.choice(english).replace('\n', '')
        if not is_credits_related(word):
            sentence += word.lower() + ' '
            
    if len(sentence) > 0:
        non_credits_training.append(sentence)

In [93]:
with open('./credits.train', 'w') as file:
    for line in credits_training:
        file.write('__label__credits' + ' ' + line + '\n')
    for line in non_credits_training:
        file.write('__label__not' + ' ' + line + '\n')

#### Train a model and save it to disk

In [94]:
def print_results(N, p, r):
    print("N\t" + str(N))
    print("P@{}\t{:.3f}".format(1, p))
    print("R@{}\t{:.3f}".format(1, r))

train_data = './credits.train'
valid_data = './credits.train'

model = fastText.train_supervised(
    input=train_data, epoch=50, lr=1.0, wordNgrams=2, verbose=2, minCount=1,
    loss='hs'
)
print_results(*model.test(valid_data))
model.save_model('./credits.bin')

model.quantize(input=train_data, qnorm=True, retrain=True, cutoff=100000)
print_results(*model.test(valid_data))
model.save_model('./credits.ftz')

N	4000
P@1	1.000
R@1	1.000
N	4000
P@1	1.000
R@1	1.000


#### Load the model and use it for inference

In [3]:
m = fastText.load_model('./credits.ftz')
print(m.predict('directed by james'.lower()))
print(m.predict('james went on and became an different person'.lower()))

(('__label__credits',), array([0.99999571]))
(('__label__not',), array([1.00000978]))
